In [54]:
# Install Kaggle
# !pip install kaggle

In [55]:
# Import necessary libraries
# import kaggle  #kaggle API
import zipfile  # for working with zip files
from zipfile import ZipFile  # for working with zip files
# from io import bytesOI
import pandas as pd
from urllib.request import urlopen
import os
import pycountry

# 1. Extracting Data
## Write code to extract the 31 country files from the emissions dataset that you are using for the project.

In [56]:
# Dataframe for sector
sector_data = {'sectorName': ['agriculture', 'buildings', 'fluorinated_gases', 'fossil_fuel_operations', 'manufacturing', 'mineral_extraction', 'power', 'waste']}
dimSector = pd.DataFrame(sector_data)
dimSector.insert(0, 'sectorID', range(1, len(dimSector) + 1))
print(dimSector)

   sectorID              sectorName
0         1             agriculture
1         2               buildings
2         3       fluorinated_gases
3         4  fossil_fuel_operations
4         5           manufacturing
5         6      mineral_extraction
6         7                   power
7         8                   waste


In [57]:
# Dataframe for subsector
subsector_data = {
    'subsectorName': [
        # Agriculture
        'cropland-fires', 'enteric-fermentation', 'manure-management',
        'other-agricultural-soil-emissions', 'rice-cultivation', 'synthetic-fertilizer-application',
        
        # Buildings
        'other-onsite-fuel-usage', 'residential-and-commercial-onsite-fuel-usage',
        
        # Fluorinated Gases
        'fluorinated-gases',
        
        # Fossil Fuel Operations
        'coal-mining', 'oil-and-gas-production-and-transport', 'oil-and-gas-refining',
        'other-fossil-fuel-operations', 'solid-fuel-transformation',
        
        # Manufacturing
        'aluminum', 'cement', 'chemicals', 'other-manufacturing', 'pulp-and-paper', 'steel',
        
        # Mineral Extraction
        'bauxite-mining', 'copper-mining', 'iron-mining', 'rock-quarrying', 'sand-quarrying',
        
        # Power
        'electricity-generation', 'other-energy-use',
        
        # Waste
        'biological-treatment-of-solid-waste-and-biogenic', 'incineration-and-open-burning-of-waste',
        'solid-waste-disposal', 'wastewater-treatment-and-discharge'
    ],
    'sectorName': [
        # Agriculture
        'agriculture', 'agriculture', 'agriculture',
        'agriculture', 'agriculture', 'agriculture',
        
        # Buildings
        'buildings', 'buildings',
        
        # Fluorinated Gases
        'fluorinated_gases',
        
        # Fossil Fuel Operations
        'fossil_fuel_operations', 'fossil_fuel_operations', 'fossil_fuel_operations',
        'fossil_fuel_operations', 'fossil_fuel_operations',
        
        # Manufacturing
        'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing',
        
        # Mineral Extraction
        'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction',
        
        # Power
        'power', 'power',
        
        # Waste
        'waste', 'waste', 'waste', 'waste'
    ]
}

dimSubSector = pd.DataFrame(subsector_data)
dimSubSector = dimSubSector.merge(dimSector, on='sectorName')
dimSubSector.insert(0, 'subsectorID', range(1, len(dimSubSector) + 1))
dimSubSector.drop(columns=['sectorName'], inplace=True)
print(dimSubSector)

    subsectorID                                     subsectorName  sectorID
0             1                                    cropland-fires         1
1             2                              enteric-fermentation         1
2             3                                 manure-management         1
3             4                 other-agricultural-soil-emissions         1
4             5                                  rice-cultivation         1
5             6                  synthetic-fertilizer-application         1
6             7                           other-onsite-fuel-usage         2
7             8      residential-and-commercial-onsite-fuel-usage         2
8             9                                 fluorinated-gases         3
9            10                                       coal-mining         4
10           11              oil-and-gas-production-and-transport         4
11           12                              oil-and-gas-refining         4
12          

In [58]:
# Combined csv will all the data
main_directory = 'data/'
dataframes = []

for root, dirs, files in os.walk(main_directory):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df)

       iso3_country           start_time             end_time  \
0               MKD  2020-01-01 00:00:00  2020-12-31 00:00:00   
1               MKD  2020-01-01 00:00:00  2020-12-31 00:00:00   
2               MLI  2020-01-01 00:00:00  2020-12-31 00:00:00   
3               MLI  2020-01-01 00:00:00  2020-12-31 00:00:00   
4               MLT  2020-01-01 00:00:00  2020-12-31 00:00:00   
...             ...                  ...                  ...   
256935          SMR  2021-01-01 00:00:00  2021-12-31 00:00:00   
256936          SMR  2021-01-01 00:00:00  2021-12-31 00:00:00   
256937          SMR  2021-01-01 00:00:00  2021-12-31 00:00:00   
256938          SMR  2021-01-01 00:00:00  2021-12-31 00:00:00   
256939          WLF  2021-01-01 00:00:00  2021-12-31 00:00:00   

                             original_inventory_sector         gas  \
0                    other-agricultural-soil-emissions         co2   
1                    other-agricultural-soil-emissions         ch4   
2        

In [59]:
# Dataframe for country
dimCountry = combined_df['iso3_country'].unique()
dimCountry = pd.DataFrame(dimCountry, columns=['countryCode'])
dimCountry.insert(0, 'countryID', range(1, len(dimCountry) + 1))

def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        return 'Unknown'

dimCountry['countryName'] = dimCountry['countryCode'].apply(get_country_name)
print(dimCountry)

     countryID countryCode                                   countryName
0            1         MKD                               North Macedonia
1            2         MLI                                          Mali
2            3         MLT                                         Malta
3            4         MMR                                       Myanmar
4            5         ABW                                         Aruba
..         ...         ...                                           ...
246        247         VAT                 Holy See (Vatican City State)
247        248         CCK                       Cocos (Keeling) Islands
248        249         CXR                              Christmas Island
249        250         SGS  South Georgia and the South Sandwich Islands
250        251         SMR                                    San Marino

[251 rows x 3 columns]


In [60]:
# Dataframe for gas
dimGas = combined_df['gas'].unique()
dimGas = pd.DataFrame(dimGas, columns=['gasName'])
dimGas.insert(0, 'gasID', range(1, len(dimGas) + 1))
print(dimGas)

   gasID     gasName
0      1         co2
1      2         ch4
2      3         n2o
3      4   co2e_20yr
4      5  co2e_100yr


In [61]:
# Dataframe for year 
dimYear = combined_df['start_time'].astype(str).str[:4].unique()
dimYear = pd.DataFrame(dimYear, columns=['yearName'])
dimYear.insert(0, 'yearID', range(1, len(dimYear) + 1))
print(dimYear)

   yearID yearName
0       1     2020
1       2     2019
2       3     2018
3       4     2017
4       5     2016
5       6     2015
6       7     2021
7       8     2022


In [62]:
# Dataframe for emission

# 2.1. Cleaning the data

In [63]:
# fix countries that were not mapped correctly
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryID,countryCode,countryName
221,222,SCG,Unknown
245,246,XKX,Unknown


In [64]:
dimCountry.loc[dimCountry['countryCode'] == 'XKX', 'countryName'] = 'Republic of Kosovo'
dimCountry.loc[dimCountry['countryCode'] == 'SCG', 'countryName'] = 'Serbia and Montenegro'
dimCountry.loc[dimCountry['countryCode'] == 'TUR', 'countryName'] = 'Turkey'

# should now have no rows
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryID,countryCode,countryName


In [65]:
# more cleaning here

# 2.2. Transforming the data

# 3. Loading the data to the data warehouse

In [66]:
# export data as CSV files
combined_df.to_csv('combined_dataframe.csv', index=False)
dimSector.to_csv('dimSector.csv', index=False)
dimCountry.to_csv('dimCountry.csv', index=False)
dimGas.to_csv('dimGas.csv', index=False)
dimSubSector.to_csv('dimSubSector.csv', index=False)
dimYear.to_csv('dimYear.csv', index=False)